# 12 - Performance & Spark Internals

Zrozumienie wewnętrznych mechanizmów Spark i optymalizacja wydajności.

**Tematy:**
- Architektura Spark: Driver, Executor, Task, Stage
- Spark UI - czytanie DAG i metryk
- Catalyst Optimizer - jak Spark optymalizuje zapytania
- Tungsten - zarządzanie pamięcią i codegen
- Predicate pushdown i column pruning
- Porównanie strategii joinów
- AQE (Adaptive Query Execution)
- Troubleshooting typowych problemów

## 1. Setup

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import time

spark = SparkSession.builder \
    .appName("12_Performance") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Spark UI: http://spark-master:4040 (lub sprawdź port w logach)
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
properties = {
    "user": "recommender",
    "password": "recommender",
    "driver": "org.postgresql.Driver"
}

ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=properties,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=properties)

## 2. Architektura Spark

```
Driver Program
  ├── SparkContext / SparkSession
  ├── DAGScheduler → dzieli job na stages
  └── TaskScheduler → wysyła tasks do executors

Cluster Manager (Standalone/YARN/K8s)
  ├── Executor 1
  │   ├── Task 1.1
  │   └── Task 1.2
  └── Executor 2
      ├── Task 2.1
      └── Task 2.2
```

### Kluczowe pojęcia:
- **Job** = jedna akcja (count, show, write)
- **Stage** = zbiór tasków bez shuffle (shuffle = granica stage)
- **Task** = operacja na jednej partycji
- **Shuffle** = wymiana danych między executors (przez sieć!)

In [ ]:
# Sprawdź konfigurację
print("=== Spark Configuration ===")
for key in ["spark.executor.memory", "spark.executor.cores", 
            "spark.driver.memory", "spark.sql.shuffle.partitions",
            "spark.sql.adaptive.enabled", "spark.default.parallelism"]:
    try:
        val = spark.conf.get(key)
    except:
        val = "not set"
    print(f"  {key} = {val}")

print(f"\n=== Cluster ===")
sc = spark.sparkContext
print(f"  Master: {sc.master}")
print(f"  App ID: {sc.applicationId}")
print(f"  Default parallelism: {sc.defaultParallelism}")

## 3. Lazy Evaluation i DAG

Spark nie wykonuje operacji od razu - buduje DAG (Directed Acyclic Graph) i wykonuje dopiero przy akcji.

In [ ]:
# Transformacje - NIE wykonują się od razu (lazy)
result = ratings \
    .filter(col("rating") >= 4.0) \
    .join(movies, "movie_id") \
    .groupBy("title") \
    .agg(count("*").alias("cnt"), avg("rating").alias("avg")) \
    .orderBy(desc("cnt"))

# Nic się nie wykonało - tylko zbudowano DAG
print("DAG zbudowany (ale nie wykonany)")
print(f"Plan has {len(result.explain(mode='simple') or '')} chars")

# Teraz AKCJA - wykonuje się cały DAG
start = time.time()
result.show(5)  # <-- TU się wykonuje!
print(f"\nCzas wykonania: {time.time() - start:.2f}s")

# Sprawdź Spark UI → Jobs tab → zobaczysz ten job

## 4. Catalyst Optimizer

Catalyst to optymalizator zapytań Spark SQL. Automatycznie optymalizuje plan wykonania.

### Fazy optymalizacji:
1. **Analysis** - rozwiązuje nazwy kolumn i typy
2. **Logical Optimization** - predicate pushdown, column pruning, constant folding
3. **Physical Planning** - wybiera strategię joina, agregacji itp.
4. **Code Generation** - Tungsten generuje bytecode

In [ ]:
# explain(mode="extended") - pokazuje wszystkie fazy
query = ratings \
    .filter(col("rating") >= 4.0) \
    .filter(col("user_id") < 1000) \
    .select("user_id", "movie_id", "rating")

query.explain(mode="extended")

In [ ]:
# Predicate pushdown - Catalyst przesuwa filtry bliżej źródła danych
# Obserwuj: filtry pojawiają się w Scan (przed odczytem danych!)

print("=== Predicate pushdown ===")
ratings.filter(col("user_id") == 42) \
    .select("movie_id", "rating") \
    .explain()

In [ ]:
# Column pruning - Catalyst czyta tylko potrzebne kolumny
# Obserwuj: w Scan pojawiają się tylko wybrane kolumny

print("=== Column pruning ===")
# Mimo że ratings ma 4 kolumny, Spark przeczyta tylko 2:
ratings.select("user_id", "rating").explain()

In [ ]:
# Constant folding - Catalyst upraszcza wyrażenia stałe
print("=== Constant folding ===")
ratings.filter(lit(1) == lit(1)).explain()  # filtr zostanie usunięty

print("\n=== Filter before join (optimization) ===")
# Catalyst sam przesuwa filtr PRZED join!
ratings.join(movies, "movie_id") \
    .filter(col("rating") >= 4.5) \
    .explain()

### Zadanie 1
Porównaj plany wykonania tych dwóch zapytań:

**A:** `ratings.filter(rating >= 4.0).join(movies, "movie_id").select("title", "rating")`

**B:** `ratings.join(movies, "movie_id").select("title", "rating").filter(col("rating") >= 4.0)`

Czy Catalyst optymalizuje B do tego samego planu co A?

In [ ]:
# Twoje rozwiązanie:


## 5. Tungsten - pamięć i codegen

Tungsten to silnik wykonawczy Spark:
- **Off-heap memory** - zarządzanie pamięcią poza JVM GC
- **Cache-aware computation** - optymalizacja pod cache CPU
- **Whole-stage codegen** - generuje optymalizowany bytecode Java

In [ ]:
# Sprawdź czy codegen jest włączony
print(f"Whole-stage codegen: {spark.conf.get('spark.sql.codegen.wholeStage', 'true')}")

# W explain() szukaj: *WholeStageCodegen* = Tungsten generuje kod
ratings.groupBy("movie_id") \
    .agg(count("*"), avg("rating")) \
    .explain()

In [ ]:
# Benchmark: codegen ON vs OFF
ratings.cache()
ratings.count()  # warm up

# Codegen ON
spark.conf.set("spark.sql.codegen.wholeStage", "true")
start = time.time()
ratings.groupBy("movie_id").agg(count("*"), avg("rating"), stddev("rating")).count()
on_time = time.time() - start

# Codegen OFF
spark.conf.set("spark.sql.codegen.wholeStage", "false")
start = time.time()
ratings.groupBy("movie_id").agg(count("*"), avg("rating"), stddev("rating")).count()
off_time = time.time() - start

# Przywróć
spark.conf.set("spark.sql.codegen.wholeStage", "true")

print(f"Codegen ON:  {on_time:.2f}s")
print(f"Codegen OFF: {off_time:.2f}s")
print(f"Codegen szybszy {off_time/on_time:.1f}x")

## 6. Porównanie strategii joinów

| Strategia | Kiedy | Koszt |
|-----------|-------|-------|
| **Broadcast Hash Join** | Mały dataset (<10MB) | Brak shuffle dużego DF |
| **Sort Merge Join** | Oba duże, sortowalne | Shuffle + sort obu |
| **Shuffle Hash Join** | Jeden mieści się w pamięci executora | Shuffle + hash table |
| **Cartesian Join** | cross join | O(n*m) - unikaj! |

In [ ]:
# Broadcast Hash Join - automatyczny (movies jest mały)
print("=== Broadcast Hash Join ===")
ratings.join(movies, "movie_id").explain()

start = time.time()
ratings.join(movies, "movie_id").count()
broadcast_time = time.time() - start
print(f"Czas: {broadcast_time:.2f}s")

In [ ]:
# Sort Merge Join - wymuś wyłączając broadcast
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

print("=== Sort Merge Join ===")
ratings.join(movies, "movie_id").explain()

start = time.time()
ratings.join(movies, "movie_id").count()
smj_time = time.time() - start
print(f"Czas: {smj_time:.2f}s")

# Przywróć
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "10485760")

In [ ]:
# Shuffle Hash Join - hint
print("=== Shuffle Hash Join ===")
ratings.hint("shuffle_hash").join(movies, "movie_id").explain()

start = time.time()
ratings.hint("shuffle_hash").join(movies, "movie_id").count()
shj_time = time.time() - start
print(f"Czas: {shj_time:.2f}s")

print(f"\n=== Porównanie ===")
print(f"Broadcast Hash Join: {broadcast_time:.2f}s")
print(f"Sort Merge Join:     {smj_time:.2f}s")
print(f"Shuffle Hash Join:   {shj_time:.2f}s")

## 7. AQE - Adaptive Query Execution

AQE (Spark 3.0+) optymalizuje plan W TRAKCIE wykonania na podstawie statystyk runtime.

### Optymalizacje AQE:
1. **Coalescing post-shuffle partitions** - łączy małe partycje po shuffle
2. **Switching join strategies** - zmienia SortMerge na Broadcast gdy dane okazują się małe
3. **Optimizing skew joins** - rozbija duże partycje

In [ ]:
print(f"AQE enabled: {spark.conf.get('spark.sql.adaptive.enabled')}")

# Porównanie: AQE ON vs OFF
# AQE ON
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", "200")  # domyślne - za dużo

start = time.time()
result_aqe = ratings.groupBy("movie_id").count()
result_aqe.count()
aqe_time = time.time() - start
aqe_partitions = result_aqe.rdd.getNumPartitions()

# AQE OFF
spark.conf.set("spark.sql.adaptive.enabled", "false")

start = time.time()
result_noaqe = ratings.groupBy("movie_id").count()
result_noaqe.count()
noaqe_time = time.time() - start
noaqe_partitions = result_noaqe.rdd.getNumPartitions()

spark.conf.set("spark.sql.adaptive.enabled", "true")

print(f"AQE ON:  {aqe_time:.2f}s, {aqe_partitions} partitions (AQE coalesced!)")
print(f"AQE OFF: {noaqe_time:.2f}s, {noaqe_partitions} partitions")

In [ ]:
# AQE skew join optimization
# Symulujemy skew: dodajmy dużo ocen dla jednego filmu
skewed_rows = spark.range(500000).select(
    lit(999999).alias("user_id"),
    lit(1).alias("movie_id"),  # Toy Story - bardzo dużo ocen
    (rand() * 5).cast("double").alias("rating"),
    current_timestamp().alias("rating_timestamp")
)

skewed_ratings = ratings.union(skewed_rows)

# Z AQE - powinno automatycznie rozwiązać skew
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")

print("Plan z AQE (szukaj SkewJoin):")
skewed_ratings.join(movies, "movie_id") \
    .groupBy("title") \
    .count() \
    .explain(mode="formatted")

## 8. Benchmark: operacje na danych

Systematyczne porównanie wydajności różnych podejść.

In [ ]:
def benchmark(name, func, runs=3):
    """Zmierz czas wykonania funkcji (średnia z N uruchomień)."""
    times = []
    for _ in range(runs):
        start = time.time()
        func()
        times.append(time.time() - start)
    avg_time = sum(times) / len(times)
    print(f"{name:<40} {avg_time:.3f}s (avg of {runs})")
    return avg_time

In [ ]:
# Benchmark: UDF vs built-in functions
from pyspark.sql.types import StringType

@udf(StringType())
def label_udf(rating):
    if rating >= 4.0: return "good"
    elif rating >= 2.5: return "ok"
    return "bad"

def label_builtin(r):
    return when(r >= 4.0, "good").when(r >= 2.5, "ok").otherwise("bad")

print("=== UDF vs Built-in ===")
benchmark("Python UDF", lambda: ratings.withColumn("l", label_udf(col("rating"))).count())
benchmark("Built-in when/otherwise", lambda: ratings.withColumn("l", label_builtin(col("rating"))).count())

In [ ]:
# Benchmark: repartition strategies before groupBy
print("=== Repartition strategies ===")

benchmark("No repartition (10 parts)",
    lambda: ratings.groupBy("movie_id").agg(avg("rating")).count())

benchmark("repartition(10, movie_id)",
    lambda: ratings.repartition(10, "movie_id").groupBy("movie_id").agg(avg("rating")).count())

benchmark("coalesce(2)",
    lambda: ratings.coalesce(2).groupBy("movie_id").agg(avg("rating")).count())

In [ ]:
# Benchmark: filter order
print("=== Filter order ===")

benchmark("Selective filter first (user_id=42)",
    lambda: ratings.filter(col("user_id") == 42).filter(col("rating") >= 4.0).count())

benchmark("Less selective filter first (rating>=4)",
    lambda: ratings.filter(col("rating") >= 4.0).filter(col("user_id") == 42).count())

# Catalyst powinien je zoptymalizować do tego samego planu!

## 9. Typowe problemy wydajnościowe i ich rozwiązania

In [ ]:
# Problem 1: Collecting do drivera
# NIGDY nie rób collect() na dużym DataFrame!

# ŹLE:
# all_data = ratings.collect()  # OOM na driverze!

# DOBRZE:
sample = ratings.limit(100).collect()  # mała ilość
# lub
result = ratings.groupBy("movie_id").count()  # agreguj w Spark

print("Tip: Zawsze agreguj/filtruj w Spark, nie ściągaj surowych danych do drivera")

In [ ]:
# Problem 2: Zbyt wiele małych plików po zapisie
# Rozwiązanie: coalesce przed zapisem

# ŹLE:
# ratings.write.parquet("/tmp/bad")  # 200 małych plików (shuffle.partitions)

# DOBRZE:
# ratings.coalesce(4).write.parquet("/tmp/good")  # 4 pliki

# Sprawdź ile plików powstanie:
print(f"Bez coalesce: {ratings.rdd.getNumPartitions()} plików")
print(f"Z coalesce(4): 4 pliki")

In [ ]:
# Problem 3: Wielokrotne obliczanie tego samego DataFrame
# Rozwiązanie: cache()

expensive_df = ratings.join(movies, "movie_id") \
    .withColumn("genre", explode(split(col("genres"), "\\|"))) \
    .groupBy("genre", "user_id") \
    .agg(avg("rating").alias("avg_rating"))

# BEZ cache - oblicza 2x
start = time.time()
expensive_df.filter(col("genre") == "Comedy").count()
expensive_df.filter(col("genre") == "Drama").count()
no_cache_time = time.time() - start

# Z cache - oblicza 1x
expensive_df.cache()
start = time.time()
expensive_df.filter(col("genre") == "Comedy").count()
expensive_df.filter(col("genre") == "Drama").count()
cache_time = time.time() - start

expensive_df.unpersist()

print(f"Bez cache (2x obliczenie): {no_cache_time:.2f}s")
print(f"Z cache (1x + 2x odczyt): {cache_time:.2f}s")

In [ ]:
# Problem 4: Cartesian join (cross join) - eksplozja danych
# Spark domyślnie blokuje implicit cross join

# To NIGDY nie powinno się zdarzyć na dużych danych:
# ratings.crossJoin(movies)  # 20M × 27K = 540 MILIARDÓW wierszy!

small_a = spark.createDataFrame([(1,), (2,), (3,)], ["a"])
small_b = spark.createDataFrame([("x",), ("y",)], ["b"])
print("Cross join na małych danych (3 × 2 = 6 wierszy):")
small_a.crossJoin(small_b).show()

## 10. Spark UI Checklist

Gdy masz problem z wydajnością, sprawdź w Spark UI:

### Jobs tab:
- Czy joby kończą się? Ile trwają?
- Który stage jest bottleneckiem?

### Stages tab:
- **Shuffle Read/Write** - duże wartości = dużo shuffla
- **Task Duration** - czy jest skew (1 task trwa 10x dłużej)?
- **GC Time** - >10% = za mało pamięci
- **Spill (Memory/Disk)** - dane nie mieszczą się w pamięci

### Storage tab:
- Cached DataFrames
- Ile pamięci zajmują

### SQL tab:
- Plan fizyczny zapytania
- Czas każdego operatora

## Zadanie końcowe

Masz ten (celowo nieefektywny) pipeline. Zoptymalizuj go i zmierz różnicę:

```python
# Wersja "naiwna"
result = ratings \
    .join(movies, "movie_id") \
    .withColumn("genre", explode(split(col("genres"), "\\|"))) \
    .filter(col("genre") == "Sci-Fi") \
    .filter(col("rating") >= 4.0) \
    .groupBy("title") \
    .agg(count("*").alias("cnt"), avg("rating").alias("avg")) \
    .filter(col("cnt") >= 100) \
    .orderBy(desc("avg"))
```

Wskazówki:
1. Kolejność filtrów
2. Broadcast hint
3. Filtrowanie na gatunku przed explode (genres LIKE '%Sci-Fi%')
4. shuffle.partitions
5. Porównaj explain() obu wersji

In [ ]:
# Wersja naiwna:
start = time.time()
naive = ratings \
    .join(movies, "movie_id") \
    .withColumn("genre", explode(split(col("genres"), "\\|"))) \
    .filter(col("genre") == "Sci-Fi") \
    .filter(col("rating") >= 4.0) \
    .groupBy("title") \
    .agg(count("*").alias("cnt"), avg("rating").alias("avg")) \
    .filter(col("cnt") >= 100) \
    .orderBy(desc("avg"))

naive.show(10, truncate=False)
naive_time = time.time() - start
print(f"Naive: {naive_time:.2f}s")
naive.explain()

In [ ]:
# Twoja zoptymalizowana wersja:


In [ ]:
ratings.unpersist()
spark.stop()